In [1]:
!pip install facebookads

In [3]:
import logging
from facebookads.api import FacebookAdsApi
from facebookads import adobjects

In [30]:
my_app_id = '<redacted>'
my_app_secret = '<redacted>'
my_access_token = '<redacted>'
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

In [32]:
ad_account = adobjects.adaccount.AdAccount('act_356288071441451')

In [33]:
honeypot = {
        "age_max": 26,
        "age_min": 14,
        "device_platforms": [
            "mobile",
        ],
        "excluded_connections": [
            {
                "id": "103185246428488",
                "name": "The ideas behind the Web"
            }
        ],
        "facebook_positions": [
            "feed"
        ],
        "flexible_spec": [
            {
                "interests": [
                    {
                        "id": "6003159378782",
                        "name": "Cats"
                    },
                    {
                        "id": "6003233049485",
                        "name": "I Love Dogs"
                    },
                    {
                        "id": "6003332344237",
                        "name": "Dogs"
                    },
                    {
                        "id": "6004037726009",
                        "name": "Pets"
                    }
                ]
            }
        ],
        "geo_locations": {
            "countries": [
                "IN",
                "ID",
                "PH",
                "BD"
            ],
            "location_types": [
                "home"
            ]
        },
        "publisher_platforms": [
            "facebook"
        ]
    }

whitelist = ['23842785871030444', '23842785736620444', '23842787744590444']

In [45]:
adsets = ad_account.get_ad_sets()
skip = True
for adset in adsets:
    if adset['id'] in whitelist: continue
    suspect_count = 0
    adset.remote_read(fields=['targeting', 'bid_amount', 'name'])
    ad_target = adset['targeting']
    if 'geo_locations' in ad_target and ad_target['geo_locations'] == honeypot['geo_locations']:
        suspect_count += 1
    if 'flexible_spec' in ad_target:
        for i in ad_target['flexible_spec'][0]['interests']:
            for o in honeypot['flexible_spec'][0]['interests']:
                if i == o:
                    suspect_count += 0.5
    if 'device_platforms' in ad_target and ad_target['device_platforms'] == honeypot['device_platforms']:
        suspect_count += 1
    suspect_count += 1. / (1 + abs(ad_target['age_max'] - honeypot['age_max']))
    suspect_count += 1. / (1 + abs(ad_target['age_min'] - honeypot['age_min']))
    if 'facebook_positions' in ad_target and ad_target['facebook_positions'] == honeypot['facebook_positions']:
        suspect_count += 1 
    if suspect_count > 4:
        logging.warning("Honeypot Triggered: %s -%s" % (adset['id'], adset['name']))

FacebookRequestError: 

  Message: Call was not successful
  Method:  GET
  Path:    https://graph.facebook.com/v2.11/23842786765440444/
  Params:  {'fields': 'targeting,bid_amount,name'}

  Status:  400
  Response:
    {
      "error": {
        "type": "OAuthException",
        "fbtrace_id": "HcgbzxeQrR4",
        "is_transient": true,
        "message": "(#17) User request limit reached",
        "code": 17
      }
    }
